# Chapter 10. 데이터 집계와 그룹 연산

- pandas는 데이터 집합을 자연스럽게 나누고 요약할 수 있는 groupby라는 유연한 방법을 제공함.
- 파이썬과 pandas의 강력한 표현력을 잘 이용하면 아주 복잡한 그룹 연산도 pandas 객체나 NumPy 배열을 받는 함수의 조합으로 해결할 수 있음.
- 해당 Chapter에서는 다음과 같은 내용을 배우게 된다.
    - 하나 이상의 키(함수, 배열, DataFrame의 컬럼 이름)를 이용해서 pandas 객체를 여러 조각으로 나누는 방법
    - 합계, 평균, 표준편차, 사용자 정의 함수 같은 그룹 요약 통계를 계산하는 방법
    - 정규화, 선형회귀, 등급 또는 부분집합 선택 같은 집단 내 변형이나 다른 조작을 적용하는 방법
    - 피벗테이블과 교차일람표를 구하는 방법
    - 변위치 분석과 다른 통게 집단 분석을 수행하는 방법

## 10.1 GroupBy 메카닉

- 그룹 연산은 `분리-적용-결합`으로 이해하면 편하다.
- 각 그룹의 색인은 다음과 같이 다양한 형태가 될 수 있으며, 모두 같은 타입일 필요도 없다.
    - 그룹을 묶을 축과 동일한 길이의 리스트나 배열
    - DataFrame의 컬럼 이름을 지칭하는 값
    - 그룹으로 묶을 값과 그룹 이름에 대응하는 사전이나 Series 객체
    - 축 색인 혹은 색인 내의 개별 이름에 대해 실행되는 함수
- 위 목록에서 마지막 세 방법은 객체를 나눌 때 사용할 배열을 생성하기 위한 방법임을 기억하자.

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.DataFrame({'key1' : ['a','a','b','b','a'],
                   'key2' : ['one','two','one','two','one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,-1.908760,0.820577
1,a,two,0.592510,1.751515
2,b,one,2.130766,-1.075104
3,b,two,0.454920,0.452624
4,a,one,2.572621,-1.424905


> 이 데이터를 key1으로 묶고 각 그룹에서 data1의 평균을 구해보자.   
> 여러가지 방법이 있지만 그 중 하나는 data1에 대해 groupby 메서드를 호출하고 key1 컬럼을 넘기는 것이다.

In [4]:
grouped = df['data1'].groupby(df['key1'])
grouped

In [5]:
grouped.mean()

key1
a    0.418790
b    1.292843
Name: data1, dtype: float64

> 만약 여러 개의 배열을 리스트로 넘겼다면 조금 다른 결과를 얻을 수 있다.

In [6]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one     0.331931
      two     0.592510
b     one     2.130766
      two     0.454920
Name: data1, dtype: float64

In [7]:
means.unstack()

key2,one,two
key1,,
a,0.331931,0.59251
b,2.130766,0.45492


> 길이만 같다면 그룹의 색인은 어떤 배열이라도 상관없다.

In [8]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])

df['data1'].groupby([states, years]).mean()

California  2005    0.592510
            2006    2.130766
Ohio        2005   -0.726920
            2006    2.572621
Name: data1, dtype: float64

> 한 그룹으로 묶을 정보는 주로 같은 DataFrame 안에서 찾게 되는데, 이 경우 컬럼 이름(문자열, 숫자 혹은 다른 파이썬 객체)을 넘겨서 그룹의 색인으로 사용할 수 있음.  

In [9]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,0.418790,0.382395
b,1.292843,-0.311240


> 위 결과를 보면 key2 컬럼이 결과에서 빠져 있는 것을 확인할 수 있다. 그 이유는 df['key2']는 숫자 데이터가 아니기 때문임.  
> 이런 컬럼은 성가신 컬럼 (nuisance column)이라고 부르며 결과에서 제외시킴.

In [10]:
df.groupby(['key1','key2']).mean()

data1     data2
key1 key2                    
a    one   0.331931 -0.302164
     two   0.592510  1.751515
b    one   2.130766 -1.075104
     two   0.454920  0.452624

> groupby를 쓰는 목적과 별개로, 일반적으로 유용한 GroupBy 메서드는 `그룹의 크기`를 담고 있는 Series를 반환하는 Size 메서드다.

In [11]:
df.groupby(['key1','key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### 10.1.1 그룹 간 순회하기

> GroupBy 객체는 이터레이션을 지원하는데, 그룹 이름과 그에 따른 데이터 묶음을 튜플로 반환함.

In [12]:
for name, group in df.groupby('key1') : 
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one -1.908760  0.820577
1    a  two  0.592510  1.751515
4    a  one  2.572621 -1.424905
b
  key1 key2     data1     data2
2    b  one  2.130766 -1.075104
3    b  two  0.454920  0.452624


> 색인이 여럿 존재하는 경우 튜플의 첫 번째 원소가 색인값이 됨.

In [13]:
for (k1,k2), group in df.groupby(['key1','key2']) :
    print((k1,k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one -1.908760  0.820577
4    a  one  2.572621 -1.424905
('a', 'two')
  key1 key2    data1     data2
1    a  two  0.59251  1.751515
('b', 'one')
  key1 key2     data1     data2
2    b  one  2.130766 -1.075104
('b', 'two')
  key1 key2    data1     data2
3    b  two  0.45492  0.452624


> 한 줄이면 그룹별 데이터를 사전형으로 쉽게 바꿔서 유용하게 사용할 수 있음.

In [13]:
pieces = dict(list(df.groupby('key1')))
pieces['b']

,key1,key2,data1,data2
2,b,one,-0.339307,1.133441
3,b,two,-0.357767,-0.492343


> groupby 메서드는 기본적으로 axis = 0 에 대해 그룹을 만드는데, 다른 축으로 그룹을 만드는 것도 가능.  
> 예를 들어 예제로 살펴본 df의 컬럼을 dtype에 따라 그룹으로 묶을 수도 있음.

In [20]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [21]:
grouped = df.groupby(df.dtypes, axis = 1)

for dtype, group in grouped :
    print(dtype)
    print(group)

float64
      data1     data2
0 -1.908760  0.820577
1  0.592510  1.751515
2  2.130766 -1.075104
3  0.454920  0.452624
4  2.572621 -1.424905
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


### 10.1.2 컬럼이나 컬럼의 일부만 선택하기

> DataFrame에서 만든 GroupBy 객체를 컬럼 이름이나 컬럼 이름이 담긴 배열로 색인하면 수집을 위해 해당 컬럼을 선택하게 됨.  
> 상단에 위치한 코드는 아래 코드의 신택틱 슈거로 두개 다 같은 값을 반환함.

In [22]:
df.groupby('key1')['data1'].mean()

key1
a    0.418790
b    1.292843
Name: data1, dtype: float64

In [23]:
df['data1'].groupby(df['key1']).mean()

key1
a    0.418790
b    1.292843
Name: data1, dtype: float64

> 대용량 데이터를 다룰 경우 소수의 컬럼만 집계하고 싶고 결과를 DataFrame으로 받고 싶다면 아래와 같이 작성한다.

In [29]:
df.groupby(['key1','key2'])[['data2']].mean()

data2
key1 key2          
a    one  -1.143534
     two   0.607255
b    one   1.133441
     two  -0.492343

> 색인으로 얻은 객체는 groupby 메서드에 `리스트`나 `배열`을 넘겼을 경우 `DataFrameGroupBy` 객체가 됨.  
> 색인으로 얻은 객체 groupby 메서드에 단일 값으로 `하나의 컬럼 이름`만 넘겼을 경우 `SeriesGroupBy` 객체가 됨.

In [32]:
s_grouped = df.groupby(['key1','key2'])['data2']   # 하나의 컬럼 이름만 넘기는 경우 Series 객체가 됨
s_grouped             

In [34]:
s_grouped.mean()

key1  key2
a     one    -1.143534
      two     0.607255
b     one     1.133441
      two    -0.492343
Name: data2, dtype: float64

In [36]:
d_grouped = df.groupby(['key1','key2'])[['data2']]  # 리스트를 넘기는 경우 DataFrame 객체가 됨
d_grouped

In [37]:
d_grouped.mean()

data2
key1 key2          
a    one  -1.143534
     two   0.607255
b    one   1.133441
     two  -0.492343

### 10.1.3 사전과 Series에서 그룹핑하기

> 그룹 정보는 배열이 아닌 형태로 존재하기도 함.

In [45]:
people = pd.DataFrame(np.random.randn(5,5),
                      columns = list('abcde'),
                      index = ['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])

people.iloc[2:3, [1,2]] = np.nan  # nan 값을 추가

people

,a,b,c,d,e
Joe,-0.667757,-0.161550,-2.624866,0.794498,-0.331984
Steve,0.262412,0.694519,0.712741,0.142722,0.116113
Wes,0.616616,NaN,NaN,-1.180413,0.220806
Jim,-0.277209,-0.772266,-1.094085,-0.766731,-1.728090
Travis,0.346855,-0.511015,0.883641,1.244979,-0.686450


> 이제 각 컬럼을 나타낼 그룹 목록이 있고, 그룹별로 컬럼의 값을 모두 더한다고 해보자.

In [47]:
mapping = {'a' : 'red', 
           'b' : 'red', 
           'c' : 'blue', 
           'd' : 'blue', 
           'e' : 'red',
           'f' : 'orange'}

> 이 사전에서 groupby 메서드로 넘길 배열을 뽑아낼 수 있지만 그냥 이 사전을 groupby 메서드로 넘기자.  
> (사용하지 않는 그룹 키도 문제없다는 것을 보이기 위해 'f'도 포함.)

In [48]:
by_column = people.groupby(mapping, axis = 1)
by_column.sum()

,blue,red
Joe,-1.830368,-1.161291
Steve,0.855463,1.073044
Wes,-1.180413,0.837423
Jim,-1.860816,-2.777565
Travis,2.128621,-0.850610


> Series에 대해서도 같은 기능을 수행할 수 잇는데, 고정된 크기의 맵이라고 보면 됨.

In [49]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [50]:
people.groupby(map_series, axis = 1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### 10.1.4 함수로 그룹핑하기

> 파이썬 함수를 사용하는 것은 사전이나 Series를 사용해서 그룹을 매핑하는 것보다 좀 더 일반적인 방법임.  
> 그룹 색인으로 넘긴 함수는 색인값 하나마다 한 번씩 호출되며, 반환값은 그 그룹의 이름으로 사용됨.  
> 예제 people에서는 DataFrame의 색인값을 사람의 이름으로 사용했는데, 만약 이름의 길이별로 그룹을 묶고 싶다면 이름의 길이가 담긴 배열을 만들어 넘기는 대신 len 함수를 넘기면 됨.

In [51]:
people.groupby(len).sum()

,a,b,c,d,e
3,-0.328349,-0.933816,-3.718951,-1.152646,-1.839268
5,0.262412,0.694519,0.712741,0.142722,0.116113
6,0.346855,-0.511015,0.883641,1.244979,-0.686450


> 내부적으로는 모두 배열로 변환되므로 함수를 배열, 사전 또는 Series와 섞어 쓰더라도 전혀 문제가 되지 않음.  

In [52]:
key_list = ['one','one','one','two','two']

people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -0.667757 -0.161550 -2.624866 -1.180413 -0.331984
  two -0.277209 -0.772266 -1.094085 -0.766731 -1.728090
5 one  0.262412  0.694519  0.712741  0.142722  0.116113
6 two  0.346855 -0.511015  0.883641  1.244979 -0.686450

### 10.1.5 색인 단계로 그룹핑하기

> 계층적으로 색인된 데이터는 축 색인 단계 중 하나를 사용해서 편리하게 집계할 수 있는 기능을 제공함.

In [61]:
columns = pd.MultiIndex.from_arrays([['US','US','US','JP','JP'],
                                     [1,3,5,1,3]],
                                     names = ['cty','tenor'])

hier_df = pd.DataFrame(np.random.randn(4,5),
                       columns = columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      0.050830  0.766557 -1.093277  0.975433 -0.754714
1     -0.818050  0.206955 -0.578485 -0.263617 -0.190442
2     -0.017335 -1.186706  2.260613  0.371841 -0.861071
3      1.814582  0.481880 -0.066682 -1.743004 -1.063897

> 이 기능을 사용하려면 level 예약어를 사용해서 레벨 번호나 이름을 넘기면 됨.

In [63]:
hier_df.groupby(level = 'cty', axis = 1).mean()

cty,JP,US
0,0.110360,-0.091963
1,-0.227029,-0.396526
2,-0.244615,0.352190
3,-1.403450,0.743260


## 10.2 데이터 집계

- 데이터 집계는 배열로부터 스칼라값을 만들어내는 모든 데이터 변환 작업을 말함.  
- 아래 표와 같이 많은 일반적인 데이터 집계는 데이터 묶음에 대한 준비된 통계를 계산해내는 최적화된 구현을 가지고 있음.
- 하지만 항상 이 메서들만 사용할 필요는 없음. 직접 고안한 집계함수를 사용하고 추가적으로 그룹 객체에 이미 정의된 메서드를 연결해서 사용하는 것도 가능. 
    - 예를 들어 quantile 메서드가 Series나 DataFrame 메서드이기 때문에 사용할 수 있다.

#### 최적화된 groupby 메서드

<details>
<summary> 최적화된 groupby 메서드</summary>
<div markdown="1">

|함수|설명|
|:--|:--|
|count|그룹에서 NA가 아닌 값의 수를 반환함.|
|sum|NA가 아닌 값들의 합을 구함.|
|mean|NA가 아닌 값들의 평균을 구함.|
|median|NA가 아닌 값들의 산술 중간값을 구함.|
|std, var| 편향되지 않은(n - 1을 분모로 하는) 표준편차와 분산|
|min, max| NA가 아닌 값들 중 최솟값과 최댓값|
|prod|NA가 아닌 값들의 곱|
|first, last|NA 가 아닌 값들 중 첫째 값과 마지막 값|
    
</div>
</details>


In [4]:
df 

,key1,key2,data1,data2
0,a,one,1.336260,-0.191734
1,a,two,0.079596,1.144844
2,b,one,-0.401077,-0.141785
3,b,two,1.865639,-1.173125
4,a,one,1.612219,0.422840


In [5]:
grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)

key1
a    1.557027
b    1.638967
Name: data1, dtype: float64

> 자신만의 데이터 집계함수를 사용하려면 배열의 aggregate나 agg메서드에 해당 함수를 넘기면 됨.

In [29]:
def peak_to_peak(arr) :
    return arr.max() - arr.min()

grouped.agg(peak_to_peak)

total_bill   tip  size   tip_pct
day  smoker                                  
Fri  No           10.29  2.00     1  0.067349
     Yes          34.42  3.73     3  0.159925
Sat  No           41.08  8.00     3  0.235193
     Yes          47.74  9.00     4  0.290095
Sun  No           39.40  4.99     4  0.193226
     Yes          38.10  5.00     3  0.644685
Thur No           33.68  5.45     5  0.193350
     Yes          32.77  3.00     2  0.151240

> describe 같은 메서드는 데이터를 집계하지 않는데도 잘 작동함을 확인할 수 있음.

In [7]:
grouped.describe()

data1                                                             \
     count      mean       std       min       25%       50%      75%   
key1                                                                    
a      3.0  1.009358  0.816935  0.079596  0.707928  1.336260  1.47424   
b      2.0  0.732281  1.602810 -0.401077  0.165602  0.732281  1.29896   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     1.612219   3.0  0.458650  0.669008 -0.191734  0.115553  0.422840   
b     1.865639   2.0 -0.657455  0.729267 -1.173125 -0.915290 -0.657455   

                          
           75%       max  
key1                      
a     0.783842  1.144844  
b    -0.399620 -0.141785

### 10.2.1 컬럼에 여러 가지 함수 적용하기

> 팁 데이터를 불러온 후 팁의 비율을 담기 위한 컬럼인 tip_pct를 추가함.

In [24]:
tips = pd.read_csv('examples/tips.csv')

In [25]:
# total_bill 에서 팁의 비율을 추가.
tips['tip_pct'] = tips['tip']/tips['total_bill']
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


> 컬럼에 따라 다른 함수를 사용해서 집계를 수행하거나 여러 개의 함수를 한 번에 적용하기를 원한다면 이를 쉽고 간단하게 수행할 수 있다.  
> 먼저 tips를 day와 smoker 별로 묶어보자.

In [26]:
grouped = tips.groupby(['day','smoker'])

> agg를 사용할 때 기술 통계는 함수 이름을 문자열로 넘기면 됨.

In [27]:
grouped_pct = grouped['tip_pct']
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

> 만일 함수 목록이나 함수 이름을 넘기면 함수 이름을 컬럼 이름으로 하는 DataFrame을 얻게 됨.

In [30]:
grouped_pct.agg(['mean','std',peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

> 이름과 함수가 담긴 (name, function) 튜플의 리스트를 넘기면 각 튜플에서 첫 번재 원소가 DataFrame에서 컬럼 이름으로 사용됨.

In [19]:
grouped_pct.agg([('foo','mean'), ('bar',np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

> DataFrame은 컬럼마다 다른 함수를 적용하거나 여러 개의 함수를 모든 컬럼에 적용할 수 있다.  
> tip_pct와 total_bill 컬럼에 대해 동일한 세 가지 통계를 계산한다고 가정하자.  
> 이는 각 컬럼을 따로 계산한 다음 concat 메서드를 이용해 keys 인자로 컬럼 이름을 넘겨 이어붙인 것과 동일함.

In [31]:
functions = ['count', 'mean', 'max']

result = grouped[['tip_pct', 'total_bill']].agg(functions)
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

> 위에서처럼 컬럼 이름과 메서드가 담긴 `튜플의 리스트`를 넘기는 것도 가능함.  
> Durchschnitt은 평균, Abweichung은 편차라는 의미의 독일어다.

In [24]:
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]

grouped[['tip_pct', 'total_bill']].agg(ftuples)

tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

> `컬럼마다 다른 함수` 를 적용하고 싶다면 agg 메서드에 컬럼 이름에 대응하는 함수가 들어있는 `사전`을 넘기면 된다.

In [25]:
grouped.agg({'tip' : np.max, 
             'size' : 'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

> 단 하나의 컬럼에라도 여러 개의 함수가 적용되었다면 DataFrame은 계층적인 컬럼을 가지게 됨.

In [32]:
grouped.agg({'tip' : ['min', 'max', 'mean', 'std'],
             'size' : 'sum'})

tip                            size
              min    max      mean       std  sum
day  smoker                                      
Fri  No      1.50   3.50  2.812500  0.898494    9
     Yes     1.00   4.73  2.714000  1.077668   31
Sat  No      1.00   9.00  3.102889  1.642088  115
     Yes     1.00  10.00  2.875476  1.630580  104
Sun  No      1.01   6.00  3.167895  1.224785  167
     Yes     1.50   6.50  3.516842  1.261151   49
Thur No      1.25   6.70  2.673778  1.282964  112
     Yes     2.00   5.00  3.030000  1.113491   40

### 10.2.2 색인되지 않은 형태로 집계된 데이터 반환하기

> 지금까지 살펴본 모든 예제에서 집계된 데이터는 유일한 그룹키 조합으로 색인되어 반환되어왔다.  
> 하지만 항상 이런 동작을 기대하는 것은 아니므로 groupby 메서드에 as_index = False를 넘겨서 색인되지 않도록 할 수 있다.  
> 물론 이렇게 하지 않고 색인된 결과에 대해 reset_index 메서드를 호출해서 같은 결과를 얻을 수 있다. 다만, as_index = False 옵션을 사용하면 불필요한 계산을 피할 수 있다.

In [33]:
tips.groupby(['day', 'smoker'], as_index = False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


## 10.3 Apply : 일반적인 분리-적용-병합
- 가장 일반적인 GroupBy 메서드의 목적은 apply임.  
- apply 메서드는 객체를 여러 조각으로 나누고, 전달된 함수를 각 조각에 일괄 적용한 후 이를 다시 합침.  

> 앞서 살펴본 팁 데이터에서 그룹별 상위 5개의 tip_pct 값을 골라서 보자.  
> 우선 특정 컬럼에서 가장 큰 값을 가지는 로우를 선택하는 함수를 바로 작성해보자.

In [34]:
def top(df, n = 5, column = 'tip_pct') :    
    return df.sort_values(by = column, ascending = False)[:n]

top(tips, n = 6)

,total_bill,tip,smoker,day,time,size,tip_pct
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
232,11.61,3.39,No,Sat,Dinner,2,0.291990
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525


> 흡연자(Smoker) 그룹에 대해 top 함수를 apply하면 다음과 같은 결과를 얻을 수 있다.  
> 아래 결과는 다음과 같은 절차를 걸쳐 출력된다.  
>    1. top 함수가 나뉘어진 DataFrame의 각 부분에 모두 적용.  
>    2. pandas.concat을 이용해서 하나로 합쳐진 다음 그룹 이름표가 붙음.   
>   3. 결과는 계층적 색인을 가지게 되고 내부 색인은 원본 DataFrame의 색인값을 가지게 됨.  

In [37]:
tips.groupby("smoker").apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     232       11.61  3.39     No   Sat  Dinner     2  0.291990
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       88        24.71  5.85     No  Thur   Lunch     2  0.236746
Yes    172        7.25  5.15    Yes   Sun  Dinner     2  0.710345
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       109       14.31  4.00    Yes   Sat  Dinner     2  0.279525

> 만일 apply 메서드로 넘길 함수가 추가적인 인자를 받는다면 함수 이름 뒤에 붙여서 넘겨주면 된다.

In [38]:
tips.groupby(['smoker','day']).apply(top, n =1 , column  = 'total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

> 앞서 groupby 객체에 describe 메서드를 호출했던 적이 있다.

In [39]:
result = tips.groupby('smoker')[['tip_pct']].describe()
result

tip_pct                                                              \
         count      mean       std       min       25%       50%       75%   
smoker                                                                       
No       151.0  0.159328  0.039910  0.056797  0.136906  0.155625  0.185014   
Yes       93.0  0.163196  0.085119  0.035638  0.106771  0.153846  0.195059   

                  
             max  
smoker            
No      0.291990  
Yes     0.710345

In [40]:
result.unstack()

                smoker
tip_pct  count  No        151.000000
                Yes        93.000000
         mean   No          0.159328
                Yes         0.163196
         std    No          0.039910
                Yes         0.085119
         min    No          0.056797
                Yes         0.035638
         25%    No          0.136906
                Yes         0.106771
         50%    No          0.155625
                Yes         0.153846
         75%    No          0.185014
                Yes         0.195059
         max    No          0.291990
                Yes         0.710345
dtype: float64

> describe 같은 메서드를 호출하면 GroupBy 내부적으로 다음과 같은 단계를 수행함.

In [41]:
f = lambda x : x.describe()
tips.groupby('smoker').apply(f)

total_bill         tip        size     tip_pct
smoker                                                      
No     count  151.000000  151.000000  151.000000  151.000000
       mean    19.188278    2.991854    2.668874    0.159328
       std      8.255582    1.377190    1.017984    0.039910
       min      7.250000    1.000000    1.000000    0.056797
       25%     13.325000    2.000000    2.000000    0.136906
       50%     17.590000    2.740000    2.000000    0.155625
       75%     22.755000    3.505000    3.000000    0.185014
       max     48.330000    9.000000    6.000000    0.291990
Yes    count   93.000000   93.000000   93.000000   93.000000
       mean    20.756344    3.008710    2.408602    0.163196
       std      9.832154    1.401468    0.810751    0.085119
       min      3.070000    1.000000    1.000000    0.035638
       25%     13.420000    2.000000    2.000000    0.106771
       50%     17.920000    3.000000    2.000000    0.153846
       75%     26.860000    3.680000    3.000000    0.195059
       max     50.810000   10.000000    5.000000    0.710345

### 10.3.1 그룹 색인 생략하기 

> 앞선 예제들에서 반환된 객체는 원본 객체의 각 조각에 대한 색인과 그룹 키가 계층적 색인으로 사용됨을 볼 수 있었음.  
> 이런 결과는 groupby 메서드에 group_keys = False 를 넘겨서 막을 수 있음.

In [42]:
tips.groupby('smoker', group_keys = False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
232,11.61,3.39,No,Sat,Dinner,2,0.291990
149,7.51,2.00,No,Thur,Lunch,2,0.266312
51,10.29,2.60,No,Sun,Dinner,2,0.252672
185,20.69,5.00,No,Sun,Dinner,5,0.241663
88,24.71,5.85,No,Thur,Lunch,2,0.236746
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525


### 10.3.2 변위치 분석과 버킷 분석
- pandas의 cut과 qcut 메서드를 사용해서 선택한 크기만큼 혹은 표본 변위치에 따라 데이터를 나눌 수 있었다.  
- 이 함수들을 groupby와 조합하면 데이터 묶음에 대해 변위치 분석이나 버킷 분석을 매우 쉽게 수행할 수 있다.  
- 임의의 데이터 묶음을 cut을 이용해 등간격 구분으로 나누어보자.

In [43]:
frame = pd.DataFrame({'data1' : np.random.randn(1000),
                      'data2' : np.random.randn(1000)})
quartiles = pd.cut(frame.data1, 4)
quartiles[:10]

0      (0.203, 1.787]
1    (-2.971, -1.381]
2    (-2.971, -1.381]
3      (0.203, 1.787]
4     (-1.381, 0.203]
5     (-1.381, 0.203]
6     (-1.381, 0.203]
7      (0.203, 1.787]
8      (1.787, 3.371]
9     (-1.381, 0.203]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-2.971, -1.381] < (-1.381, 0.203] < (0.203, 1.787] < (1.787, 3.371]]

> cut에서 반환된 Categorical 객체는 바로 groupby로 넘길 수 있다.  
> 그러므로 data2 컬럼에 대한 몇 가지 통계를 다음과 같이 계산할 수 있다.

In [44]:
def get_stats(group) : 
    return {'min' : group.min(),
            'max' : group.max(),
            'count' : group.count(), 
            'mean' : group.mean()}

grouped = frame.data2.groupby(quartiles)    # quartiles는 frame안에 있는 배열이 아니므로 frame.groupby(quartiles)로 groupby를 할 수 없음! 
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-2.971, -1.381]",-2.160400,2.011165,88.0,-0.144329
"(-1.381, 0.203]",-3.638611,2.922929,488.0,-0.022641
"(0.203, 1.787]",-2.727984,2.373485,397.0,-0.072920
"(1.787, 3.371]",-1.842143,2.623581,27.0,0.036921


> 표본 변위치에 기반하여 크기가 같은 버킷을 계산하려면 qcut을 사용한다.  
> labels = False를 넘기면 변위치 숫자를 구할 수 있다.

In [45]:
grouping = pd.qcut(frame.data1, 10, labels = False)

grouped = frame.data2.groupby(grouping)
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
0,-2.160400,2.011165,100.0,-0.166612
1,-3.638611,2.313730,100.0,-0.148575
2,-1.801247,2.055202,100.0,-0.153270
3,-2.825552,2.740370,100.0,-0.036695
4,-2.114450,2.922929,100.0,0.161845
5,-2.089558,2.920856,100.0,0.153673
6,-2.727984,1.839289,100.0,0.010669
7,-2.457763,1.994723,100.0,-0.129396
8,-2.001674,1.800016,100.0,-0.156903


### 10.3.3 예제 : 그룹에 따른 값으로 결측값 채우기 
- 누락된 데이터를 정리할 때면 어떤 경우에는 dropna를 사용해서 데이터를 살펴보고 걸러내기도 함.  
- 하지만 어떤 경우에는 누락된 값을 고정된 값이나 혹은 데이터로부터 도출된 어떤 값으로 채우고 싶을 때도 있다. 
- 이런 경우 fillna 메서드를 사용함.
- 아래는 누락된 값을 평균값으로 대체하는 예제

In [46]:
s = pd.Series(np.random.randn(6))
s[::2] = np.nan
s

0         NaN
1   -2.101965
2         NaN
3    0.866285
4         NaN
5   -0.706898
dtype: float64

In [47]:
s.fillna(s.mean())

0   -0.647526
1   -2.101965
2   -0.647526
3    0.866285
4   -0.647526
5   -0.706898
dtype: float64

> 그룹별로 채워 넣고 싶은 값이 다를 경우 apply 함수를 사용해 각 그룹에 fillna를 적용하면 된다.

In [48]:
states = ['Ohio', 'New York', 'Vermont', 'Florida',
          'Oregon', 'Nevada', 'California', 'Idaho']
group_key = ['East'] * 4 + ['West'] * 4

data = pd.Series(np.random.rand(8), index = states)
data

Ohio          0.459333
New York      0.915046
Vermont       0.736742
Florida       0.551162
Oregon        0.063674
Nevada        0.636531
California    0.893293
Idaho         0.220555
dtype: float64

In [49]:
data[['Vermont','Nevada','Idaho']] = np.nan
data

Ohio          0.459333
New York      0.915046
Vermont            NaN
Florida       0.551162
Oregon        0.063674
Nevada             NaN
California    0.893293
Idaho              NaN
dtype: float64

In [50]:
data.groupby(group_key).mean()

East    0.641847
West    0.478483
dtype: float64

> 다음과 같이 누락된 값을 그룹의 평균값으로 채울 수 있다.

In [51]:
fill_mean = lambda g : g.fillna(g.mean())

data.groupby(group_key).apply(fill_mean)

Ohio          0.459333
New York      0.915046
Vermont       0.641847
Florida       0.551162
Oregon        0.063674
Nevada        0.478483
California    0.893293
Idaho         0.478483
dtype: float64

> 아니면 그룹에 따라 미리 정의된 다른 값을 채워 넣어야 하는 경우, 각 그룹 내부적으로 가진 name이라는 속성을 이용하자.

In [52]:
fill_values = {'East' : 0.5,
               "West" : -1}

fill_func = lambda g : g.fillna(fill_values[g.name])

data.groupby(group_key).apply(fill_func)

Ohio          0.459333
New York      0.915046
Vermont       0.500000
Florida       0.551162
Oregon        0.063674
Nevada       -1.000000
California    0.893293
Idaho        -1.000000
dtype: float64

### 10.3.4 예제 : 랜덤 표본과 순열

- 대용량의 데이터를 몬테카를로 시뮬레이션이나 다른 애플리케이션에서 사용하기 위해 랜덤 표본을 봅아낸다고 해보자.  
- 뽑아내는 방법은 여러 가지가 있는데, 여기서는 Series의 sample 메서드를 사용하자.
- 예시를 위해 트럼프 카드 덱을 만들어보자.

In [53]:
# 하트, 스페이드, 클럽, 다이아몬드
suits = ['H', 'S', 'C', 'D']
card_val =(list(range(1,11)) + [10]*3)*4
base_names = ['A'] + list(range(2,11)) + ['J', 'K', 'Q']
cards = []

for suit in ['H','S','C','D'] :
    cards.extend(str(num) + suit for num in base_names)

deck = pd.Series(card_val, index = cards)
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

> 5장의 카드를 뽑기 위한 코드 작성

In [54]:
def draw(deck, n = 5) :
    return deck.sample(n)

draw(deck)

9H      9
4S      4
5C      5
4C      4
10D    10
dtype: int64

> 각 세트 (하트, 스페이드, 클럽, 다이아몬드) 별로 2장의 카드를 무작위로 뽑고 싶다고 가정.  
> 세트는 각 카드 이름의 마지막 글자이므로 이를 이용해 그룹을 나누고 apply를 사용하자. 

In [55]:
get_suit = lambda card : card[-1]  # 마지막 글자가 세트 

deck.groupby(get_suit).apply(draw, n = 2)

C  3C      3
   6C      6
D  5D      5
   4D      4
H  10H    10
   7H      7
S  6S      6
   AS      1
dtype: int64

> 아래와 같은 방법으로 각 세트별 2장의 카드를 무작위로 뽑을 수도 있다.

In [56]:
deck.groupby(get_suit, group_keys = False).apply(draw, n = 2)

6C     6
3C     3
6D     6
4D     4
KH    10
3H     3
6S     6
JS    10
dtype: int64

### 10.3.5 예제 : 그룹 가중 평균과 상관관계
- groupby의 나누고 적용하고 합치는 패러다임에서 (그룹 가중 평균과 같은) DataFrame의 컬럼 간 연산이나 두 Series 간의 연산은 일상적인 일이다.
- 예를 들어 그룹 키와 값 그리고 어떤 가중치를 갖는 다음 데이터 묶음을 살펴보자.

In [57]:
df = pd.DataFrame({'category' : list('aaaabbbb'),
                   'data' : np.random.randn(8),
                   'weights' : np.random.randn(8)})
df

,category,data,weights
0,a,0.447141,-1.996237
1,a,0.840076,0.212000
2,a,0.372974,0.343870
3,a,0.998538,-1.776436
4,b,1.232246,-0.340581
5,b,0.214467,0.585257
6,b,-0.568154,-1.586111
7,b,-0.193992,0.559277


In [58]:
grouped = df.groupby('category')

get_wavg = lambda g : np.average(g['data'], weights = g['weights'])

grouped.apply(get_wavg)

category
a    0.733674
b   -0.637338
dtype: float64

> 좀 더 복잡한 예제로 야후!파이낸스에서 가져온 몇몇 주식과 S&P500 지수 (종목 코드 SPX)의 종가 데이터를 살펴보자.

In [59]:
close_px = pd.read_csv('examples/stock_px_2.csv', parse_dates = True, index_col = 0)
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


In [61]:
close_px[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


> 퍼센트 변화율로 일일 수익률을 계산하여 연간 SPX 지수와의 상관관계를 살펴보는 일은 흥미로울 수 있다.   
> 우선 'SPX'컬럼과 다른 컬럼의 상관관계를 계산하는 함수를 만든다.

In [62]:
spx_corr = lambda x : x.corrwith(x['SPX'])

> 그리고 pct_change 함수를 이용해서 close_px의 퍼센트 변화율을 계산한다.

In [63]:
rets = close_px.pct_change().dropna()

> 마지막으로 각 datetime에서 연도 속성만 반환하는 한줄 짜리 함수를 이용해서 연도별 종목별 퍼센트 변화율과 SPX의 상관계수를 구한다.

In [64]:
get_year = lambda x : x.year

corr_by_year = rets.groupby(get_year).apply(spx_corr)
corr_by_year

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


> 물론 두 컬럼 간의 상관관계를 계산하는 것도 가능하다.  
> 다음은 애플과 마이크로소프트 주가의 연간 상관관계다.

In [65]:
rets.groupby(get_year).apply(lambda g : g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

> ### 10.3.6 예제 : 그룹상의 선형회귀

- pandas 객체나 스칼라값을 반환하기만 한다면 groupby를 좀 더 복잡한 그룹상의 통계 분석을 위해 사용할 수 있다.  
- 예를 들어 계량경제 라이브러리인 statsmodels를 사용해서 regress라는 함수를 작성하고 각 데이터 묶음마다 최소제곱 (OLS)로 회귀를 수행할 수 있다.

In [ ]:
# !pip install statsmodels

In [66]:
import statsmodels.api as sm

def regress(data, yvar, xvars) :
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y,X).fit()
    return result.params

> 이제 SPX 수익률에 대한 애플 (AAPL) 주식의 연간 선형회귀는 다음과 같이 수행할 수 있다.

In [74]:
rets.groupby(get_year).apply(regress, 'AAPL', ['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


## 10.4 피벗테이블과 교차일람표

- 피벗테이블은 스프레드시트 프로그램과 그 외 다른 데이터 분석 소프트웨어에서 흔히 볼 수 있는 데이터 요약화 도구.
- 피벗테이블은 데이터를 하나 이상의 키로 수집해서 어떤 키는 로우에, 어떤 키는 컬럼에 나열해서 데이터를 정렬한다.
- pandas에서 피벗 테이블은 이 장에서 설명했던 groupby 기능을 사용해서 계층적 색인을 활용한 재형성 연산을 가능하게 해준다.
- DataFrame에는 pivot_table 메서드가 있는데 이는 pandas 모듈의 최상위 함수로도 존재함. (pandas.pivot_table).
- groupby를 위한 편리한 인터페이스를 제공하기 위해 pivot_table은 마진이라고 하는 부분합을 추가할 수 있는 기능을 제공함.

> 팁 데이터로 돌아가서 요일(day)과 흡연자(smoker) 집단에서 평균(pivot_table의 기본 연산)을 구해보자.

In [75]:
tips.pivot_table(index = ['day','smoker'])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

> 이는 groupby를 사용해서 쉽게 구할 수 있는데, 이제 tip_pct와 size에 대해서만 집계를 하고 날짜(time)별로 그룹지어보자.

In [76]:
tips.pivot_table(['tip_pct', 'size'], index = ['time','day'],
                 columns = 'smoker')
# tips.groupby(["time","day","smoker"])[['size','tip_pct']].mean().unstack()  # 위와 동일한 코드 

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

> 이 테이블은 margins = True를 넘겨서 부분합을 포함하도록 확장할 수 있음.  
> 그렇게 하면 `All컬럼`과 `All 로우`가 추가되어 단일 줄 안에서 그룹 통계를 얻을 수 있다.  
> 여기서 All 값은 흡연자와 비흡연자를 구분하지 않은 평균값 (All 컬럼)이거나 로우에서 두 단계를 묶은 그룹의 평균값(All 로우)이다.

In [80]:
tips.pivot_table(['tip_pct', 'size'], index = ['time', 'day'],
                 columns = 'smoker', margins = True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

> 다른 집계 함수를 사용하려면 그냥 aggfunc로 넘기면 됨.  
> 예를 들어 'count'나 len 함수는 그룹 크기의 교차일람표 (총 개수나 빈도)를 반환함.

In [84]:
tips.pivot_table('tip_pct', index = ['time', 'smoker'], columns = 'day',
                 aggfunc = len, margins = True)

day             Fri   Sat   Sun  Thur    All
time   smoker                               
Dinner No       3.0  45.0  57.0   1.0  106.0
       Yes      9.0  42.0  19.0   NaN   70.0
Lunch  No       1.0   NaN   NaN  44.0   45.0
       Yes      6.0   NaN   NaN  17.0   23.0
All            19.0  87.0  76.0  62.0  244.0

> 만약 어떤 조합이 비어 있다면 (혹은 NA값) fill_value를 넘길 수도 있음.

In [85]:
tips.pivot_table('tip_pct', index = ['time', 'size', 'smoker'],
                 columns = 'day', aggfunc = 'mean', fill_value = 0)

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
            Yes     0.000000  0.144995  0.152660  0.000000
       4    No      0.000000  0.150096  0.148143  0.000000
            Yes     0.117750  0.124515  0.193370  0.000000
       5    No      0.000000  0.000000  0.206928  0.000000
            Yes     0.000000  0.106572  0.065660  0.000000
       6    No      0.000000  0.000000  0.103799  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843
       3    No      0.187735  0.000000  0.000000  0.084246
            Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

#### pivot_table 옵션

<details>
<summary> pivot_table 옵션 </summary>
<div markdown="1">

|함수|설명|
|:--|:--|
|values|집계하려는 컬럼 이름 혹은 이름의 리스트, 기본적으로 모든 숫자 컬럼을 집계함.|
|index|만들어지는 피벗테이블의 로우를 그룹으로 묶을 컬럼 이름이나 그룹 키|
|columns|만들어지는 피벗테이블의 컬럼을 그룹으로 묶을 컬럼 이름이나 그룹 키|
|aggfunc|집계함수나 함수 리스트, 기본값으로 'mean'이 사용된다. groupby 컨텍스트 안에서 유효한 어떤 함수라도 가능하다.|
|fill_value|결과 테이블에서 누락된 값을 대체하기 위한 값|
|dropna|True인 경우 모든 항목이 NA인 `컬럼`은 포함하지 않는다.|
|margins|부분합이나 총계를 담기 위한 로우/컬럼을 추가할지 여부, 기본값은 False|
    
</div>
</details>


### 10.4.1 교차일람표

> 교차일람표 (또는 교차표)는 그룹 빈도를 계산하기 위한 피벗테이블의 특수한 경우다.

In [87]:
data = pd.DataFrame({'Sample' : list(range(1,11)),
                     'Nationality' : ['USA','Japan','USA','Japan','Japan',
                                      'Japan','USA','USA','Japan','USA'],
                     'Handedness' : ['Right-handed','Left-handed','Right-handed','Right-handed','Left-handed',
                                     'Right-handed','Right-handed','Left-handed','Right-handed','Right-handed']})
data

,Sample,Nationality,Handedness
0,1,USA,Right-handed
1,2,Japan,Left-handed
2,3,USA,Right-handed
3,4,Japan,Right-handed
4,5,Japan,Left-handed
5,6,Japan,Right-handed
6,7,USA,Right-handed
7,8,USA,Left-handed
8,9,Japan,Right-handed
9,10,USA,Right-handed


> 설문 분석의 일부로서 이 데이터를 성별과 잘 쓰는 손에 따라 요약해보자.  
> 이를 위해 pivot_table 메서드를 사용할 수 있지만 pandas.crosstab 함수가 훨씬 더 편리하다.

In [100]:
pd.crosstab(data.Nationality, data.Handedness, margins = True)

# data.pivot_table(index = "Nationality", columns = "Handedness", aggfunc = 'count', margins = True)   # 위와 같은 코드 

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


> crosstab 함수의 처음 두 인자는 배열이나 Series 혹은 배열의 리스트가 될 수 있다.

In [101]:
pd.crosstab([tips.time, tips.day], tips.smoker, margins = True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244